In [1]:
#! pip install transformers datasets

In [2]:
import pandas as pd
from scipy.special import result
from sklearn.model_selection import train_test_split

In [3]:
df=pd.read_csv("politifact_fixed.csv")

In [4]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

C:\Users\Dell\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
df.head()

,Unnamed: 0,type,content_sentance
0,0,0,At a Glance Indicators MicroData Publications ...
1,1,0,CQ Username Password Stay signed in Forgot you...
2,2,0,Login | Contact Us | Site Map | Paid archives ...
3,3,0,Home Democratic Leaders Say House Democrats A...
4,4,0,OMB Home White House Website THE NATION’S FISC...


In [6]:
df=df.drop(columns='Unnamed: 0')

In [7]:
for i in df["content_sentance"]:
  if isinstance(i,int):
    print(i)

In [8]:
df=df.dropna()

In [9]:
df.head()

,type,content_sentance
0,0,At a Glance Indicators MicroData Publications ...
1,0,CQ Username Password Stay signed in Forgot you...
2,0,Login | Contact Us | Site Map | Paid archives ...
3,0,Home Democratic Leaders Say House Democrats A...
4,0,OMB Home White House Website THE NATION’S FISC...


In [10]:
from datasets import Dataset

In [11]:
!pip install dataset


[notice] A new release of pip is available: 23.2.1 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [12]:
!pip install datasets


[notice] A new release of pip is available: 23.2.1 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [13]:
df.columns = ['label', 'text']

In [14]:
df

,label,text
0,0,At a Glance Indicators MicroData Publications ...
1,0,CQ Username Password Stay signed in Forgot you...
2,0,Login | Contact Us | Site Map | Paid archives ...
3,0,Home Democratic Leaders Say House Democrats A...
4,0,OMB Home White House Website THE NATION’S FISC...
...,...,...
532,1,Facebook USPOLN U.S Political News HOME POLITI...
533,1,Daily USA Update Menu News Politics 2016 Elect...
534,1,GENIUS Sign Up Sign In | | | Facebook Twitter ...
535,1,Home Latest News Top Stories Trending News Gle...


In [15]:
dataset = Dataset.from_pandas(df,preserve_index=False)

In [47]:
train_test_split = dataset.train_test_split(test_size=0.2, seed=42)

In [17]:
#test_unsupervised_split = train_test_split['test'].train_test_split(test_size=0.5, seed=42)

In [18]:
from datasets import DatasetDict

In [48]:
dataset_dict = DatasetDict({
    'train': train_test_split['train'],
    'test': train_test_split['test'],
})

In [49]:
dataset_dict

DatasetDict({
    train: Dataset({
        features: ['label', 'text'],
        num_rows: 412
    })
    test: Dataset({
        features: ['label', 'text'],
        num_rows: 103
    })
})

In [50]:
def preprocess_function(examples):
    if isinstance(examples["text"], str):
        return tokenizer(examples["text"], truncation=True)
    else:
        return tokenizer("N/A", truncation=True)

In [51]:
df_tokenized = dataset_dict.map(preprocess_function)

Map: 100%|██████████| 103/103 [00:00<00:00, 142.59 examples/s]


In [52]:
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer, return_tensors="tf")

In [53]:
import numpy as np

In [25]:
!pip install evaluate


[notice] A new release of pip is available: 23.2.1 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [26]:
import evaluate

accuracy = evaluate.load("accuracy")

In [54]:
def compute_metrics(eval_pred):

    predictions, labels = eval_pred

    predictions = np.argmax(predictions, axis=1)

    return accuracy.compute(predictions=predictions, references=labels)

In [55]:
from transformers import create_optimizer
import tensorflow as tf
tf.config.experimental_run_functions_eagerly(True)

batch_size = 16
num_epochs = 5
batches_per_epoch = len(df_tokenized["train"])
total_train_steps = int(batches_per_epoch * num_epochs)
optimizer, schedule = create_optimizer(init_lr=2e-5, num_warmup_steps=0, num_train_steps=total_train_steps)

In [56]:
id2label = {0: "Real", 1: "Fake"}
label2id = {"Real": 0, "Fake": 1}

In [57]:
from transformers import TFAutoModelForSequenceClassification

model = TFAutoModelForSequenceClassification.from_pretrained(
    "distilbert/distilbert-base-uncased", num_labels=2, id2label=id2label, label2id=label2id
)

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFDistilBertForSequenceClassification: ['vocab_layer_norm.weight', 'vocab_transform.bias', 'vocab_projector.bias', 'vocab_transform.weight', 'vocab_layer_norm.bias']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Some weights or buffers of the TF 2.0 model TFDistilBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['pre_classifier.weight', 'pre_classifier.bias', 'classifier.weight', 'classifier.bias']
You should 

In [31]:
!pip install datasets


[notice] A new release of pip is available: 23.2.1 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [58]:
tf_train_set = model.prepare_tf_dataset(
    df_tokenized["train"],
    shuffle=True,
    batch_size=16,
    collate_fn=data_collator,
)

tf_validation_set = model.prepare_tf_dataset(
    df_tokenized["test"],
    shuffle=False,
    batch_size=16,
    collate_fn=data_collator,
)

C:\Users\Dell\AppData\Local\Programs\Python\Python310\lib\site-packages\tensorflow\python\data\ops\structured_function.py:258: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  warnings.warn(


In [59]:
import tensorflow as tf

model.compile(optimizer=optimizer,metrics=["accuracy"])

In [61]:
results=model.fit(x=tf_train_set, validation_data=tf_validation_set, epochs=3)

Epoch 1/3
25/25 [==============================] - 307s 12s/step - loss: 0.6190 - accuracy: 0.6300 - val_loss: 0.5073 - val_accuracy: 0.8932
Epoch 2/3
25/25 [==============================] - 295s 12s/step - loss: 0.3994 - accuracy: 0.8750 - val_loss: 0.2603 - val_accuracy: 0.9223
Epoch 3/3
25/25 [==============================] - 287s 12s/step - loss: 0.2004 - accuracy: 0.9475 - val_loss: 0.2373 - val_accuracy: 0.9223


In [63]:
print(results.history)

{'loss': [0.6189813017845154, 0.3994278013706207, 0.2004183530807495], 'accuracy': [0.6299999952316284, 0.875, 0.9474999904632568], 'val_loss': [0.5072781443595886, 0.2603135108947754, 0.23729723691940308], 'val_accuracy': [0.893203854560852, 0.9223300814628601, 0.9223300814628601]}
